In [1]:
%cd ..

G:\My Drive\0-Research-Project\MedStar\MS_CODE\FieldNN


# Introduction


This is the for 

* module `fieldnn.basicnn.reducer` module
* module `fieldnn.configfn.reducerfn` module
* module `fieldnn.basicnn.merger` module
* module `fieldnn.configfn.mergerfn` module

# PreCode

In [2]:
import os
from recfldgrn.datapoint import load_df_data_from_folder
from fieldnn.utils.layerfn import traverse, convert_relational_list_to_numpy

###################### take this as given
batch_PID_order = ['P1', 'P4', 'P5', 'P6']
######################

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)

l = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
# l

recfldgrn_list =['P@age-AgeNumeDftGrn',
                 'P@basicInfo-basicInfoDftGrn',
                 
                 'EC@BasicInfo-BasicDftGrn',
                 'EC@DT_min-DTDftGrn',
                 
                 'A1C@DT-DTDftGrn',
                 'A1C@V-A1CNumeDftGrn',
                 
                 'Diag@DT-DTDftGrn',
                 'Diag@Value-DiagDftGrn',
                 
                 'PNSectSent@Sentence-Tk@TknzLLMGrn']

data/ProcData/TensorFolder/Task2YearXXX


In [3]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)
    print(tensor_folder)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx
    
for k, v in batch_rfg.items(): print(k, v.shape)

data/ProcData/TensorFolder/Task2YearXXX\P@age-AgeNumeDftGrn
data/ProcData/TensorFolder/Task2YearXXX\P@basicInfo-basicInfoDftGrn
data/ProcData/TensorFolder/Task2YearXXX\EC@BasicInfo-BasicDftGrn
data/ProcData/TensorFolder/Task2YearXXX\EC@DT_min-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX\A1C@DT-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX\A1C@V-A1CNumeDftGrn
data/ProcData/TensorFolder/Task2YearXXX\Diag@DT-DTDftGrn
data/ProcData/TensorFolder/Task2YearXXX\Diag@Value-DiagDftGrn
data/ProcData/TensorFolder/Task2YearXXX\PNSectSent@Sentence-Tk@TknzLLMGrn
B-P@age-AgeNumeDftGrn_wgt (4, 19)
B-P@basicInfo-basicInfoDftGrn_idx (4, 2)
B-P-EC@BasicInfo-BasicDftGrn_idx (4, 25, 2)
B-P-EC@DT_min-DTDftGrn_idx (4, 25, 7)
B-P-EC-A1C@DT-DTDftGrn_idx (4, 25, 1, 7)
B-P-EC-A1C@V-A1CNumeDftGrn_wgt (4, 25, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (4, 25, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (4, 25, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 25, 1, 14, 121, 11)


In [4]:
full_recfldgrn_list = [k for k in batch_rfg]
full_recfldgrn_list

['B-P@age-AgeNumeDftGrn_wgt',
 'B-P@basicInfo-basicInfoDftGrn_idx',
 'B-P-EC@BasicInfo-BasicDftGrn_idx',
 'B-P-EC@DT_min-DTDftGrn_idx',
 'B-P-EC-A1C@DT-DTDftGrn_idx',
 'B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@DT-DTDftGrn_idx',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [5]:
import torch 

RECFLD_TO_TENSOR = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
   
    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    info_dict = {'holder': holder, 'holder_wgt': holder_wgt}
    
    RECFLD_TO_TENSOR[full_recfldgrn] = info_dict
    

In [6]:
from fieldnn.dataflowfn.embedflowfn import get_EmbeddingBlock_DataFlow

from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List

############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


default_SubUnitName = 'E'
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 




df_SubUnit = get_EmbeddingBlock_DataFlow(full_recfldgrn_list, default_SubUnitName)

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)
df_SubUnit['SubUnit_BasicNN_List'] = s
s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s



s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,E,[B-P@age-AgeNumeDftGrn_wgt],B-P@age-AgeNumeDft,3,[expander-NumeEmbed],"[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ...","[{'nn_type_nn_name': 'expander-NumeEmbed', 'Ba..."
1,E,[B-P@basicInfo-basicInfoDftGrn_idx],B-P@basicInfo-basicInfoDft,3,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, 'Male': 1...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
2,E,[B-P-EC@BasicInfo-BasicDftGrn_idx],B-P-EC@BasicInfo-BasicDft,4,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, 'X': 1, '...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
3,E,[B-P-EC@DT_min-DTDftGrn_idx],B-P-EC@DT_min-DTDft,4,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
4,E,[B-P-EC-A1C@DT-DTDftGrn_idx],B-P-EC-A1C@DT-DTDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
5,E,[B-P-EC-A1C@V-A1CNumeDftGrn_wgt],B-P-EC-A1C@V-A1CNumeDft,5,[expander-NumeEmbed],"[{'vocab_tokenizer': [1, 1, 1, 1, 1, 1, 1, 1, ...","[{'nn_type_nn_name': 'expander-NumeEmbed', 'Ba..."
6,E,[B-P-EC-Diag@DT-DTDftGrn_idx],B-P-EC-Diag@DT-DTDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
7,E,[B-P-EC-Diag@Value-DiagDftGrn_idx],B-P-EC-Diag@Value-DiagDft,5,[expander-CateEmbed],"[{'vocab_tokenizer': {'_padding': 0, '_missing...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
8,E,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzL...,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,7,[expander-LLMEmbed],[{'vocab_tokenizer': PreTrainedTokenizerFast(n...,"[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


In [7]:
from fieldnn.module.embedblock import EmbedBlockLayer

EmbedBlock = EmbedBlockLayer(df_SubUnit)

RECFLD_TO_EMBEDTESNOR = EmbedBlock(RECFLD_TO_TENSOR)

for k, v in RECFLD_TO_EMBEDTESNOR.items():
    print(k, v['info'].shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


# Get Input

In [8]:

for full_recfld, info_dict in RECFLD_TO_EMBEDTESNOR.items():
    print(full_recfld, info_dict['info'].shape)

B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


# Reducer: BasicNN Name 

## ReducerSum

In [22]:
import torch

class ReduceSumLayer(torch.nn.Module):
    def __init__(self):
        super(ReduceSumLayer, self).__init__()   

    def forward(self, info, leng_mask):
        # (bs, xxx, l, dim) --> (bs, xxx, dim)
        info = torch.sum(info, -2)
        return info

## ReducerMean

In [10]:
import torch

class ReduceMeanLayer(torch.nn.Module):
    def __init__(self):
        super(ReduceMeanLayer, self).__init__()
  
    def forward(self, info, leng_mask):
        leng = (leng_mask == 0).sum(-1).unsqueeze(-1).float()
        leng[leng == 0.] = 1.0 # change pad to any non-zeros to be dominators.
        info = torch.sum(info, -2) # (bs, xxx, l, dim) --> (bs, xxx, dim)
        info = info/leng           # (bs, xxx, dim)    --> (bs, xxx, dim)
        return info


## ReducerMax

In [11]:
import torch

class RecuderMaxLayer(torch.nn.Module):
    def __init__(self):
        super(RecuderMaxLayer, self).__init__()

    def forward(self, info, leng_mask):
        info = info.masked_fill(leng_mask.unsqueeze(-1), -10000) # double check this.
        a, b = info.max(-2) # not necessary, all the values could be smaller than 0.
        return a
    

## ReducerCat

In [12]:

import math

class ConcatenateLayer(torch.nn.Module):
    def __init__(self):
        super(ConcatenateLayer, self).__init__()

    def forward(self, info, leng_mask):
        l, dim = info.shape[-2:]
        info = info.view(*info.shape[:-2],  l*dim)
        return info   


# Reducer: Basic NN Type

## Module

In [18]:
import torch

# from fieldnn.basicnn.reducer.rdcmean import ReduceMeanLayer
# from fieldnn.basicnn.reducer.rdcsum import ReduceSumLayer
# from fieldnn.basicnn.reducer.rdcmax import RecuderMaxLayer
# from fieldnn.basicnn.reducer.rdccat import ConcatenateLayer

# from .rdcmean import ReduceMeanLayer
# from .rdcsum import ReduceSumLayer
# from .rdcmax import RecuderMaxLayer
# from .rdccat import ConcatenateLayer


class Reducer_Layer(torch.nn.Module):
    def __init__(self, input_names_nnlvl, output_name_nnlvl, reducer_layer_para):
        super(Reducer_Layer, self).__init__()
        
        # Part 0: Meta
        # here input_names and out_tensor just the tensor name, 
        # they are not the real tensors
        # intead, the info_dict contains the corresponding real tensors.
        assert len(input_names_nnlvl) == 1
        self.input_names_nnlvl = input_names_nnlvl
        self.input_name_nnlvl = input_names_nnlvl[0]
        
        # output_name should be generated from the input_names
        self.output_name_nnlvl = output_name_nnlvl
        # self.output_name = output_name

        # the input feature dim size and output feature dim size
        self.input_size = reducer_layer_para['input_size']
        self.output_size = reducer_layer_para['output_size']

        # Part 1: NN
        nn_name, nn_para = reducer_layer_para['nn_name'], reducer_layer_para['nn_para']
        if nn_name.lower() == 'mean':
            self.reducer = ReduceMeanLayer()
        elif nn_name.lower() == 'sum':
            self.reducer = ReduceSumLayer()
        elif nn_name.lower() == 'max':
            self.reducer = RecuderMaxLayer()
        elif nn_name.lower() == 'concat':
            self.reducer = ConcatenateLayer()
            # TODO: need to assert something
            assert self.output_size % self.input_size == 0
        else:
            raise ValueError(f'There is no layer for "{nn_name}"')
            
        # Part 2: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in reducer_layer_para['postprocess'].items():
            if method == 'activator':
                activator = config
                if activator.lower() == 'relu': 
                    self.postprocess[method] = torch.nn.ReLU()
                elif activator.lower() == 'tanh': 
                    self.postprocess[method] = torch.nn.Tanh()
                elif activator.lower() == 'gelu':
                    self.postprocess[method] = torch.nn.GELU()
            elif method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.output_size, **config)
            
    def forward(self, input_names_nnlvl, INPUTS_TO_INFODICT):
        # information preparation.
        # 'INPUTS_TO_INFODICT` will come from SubUnit Layer.
        assert len(input_names_nnlvl) == 1
        input_name_nnlvl = input_names_nnlvl[0]
        assert self.input_name_nnlvl == input_name_nnlvl
        
        info_dict = INPUTS_TO_INFODICT[input_name_nnlvl]
        holder, info = info_dict['holder'], info_dict['info']
        
        # print(holder.shape, info.shape)
        # the following part is the data proprocessing
        leng_mask = holder == 0
        info = self.reducer(info, leng_mask)
        
        for name, layer in self.postprocess.items():
            info = layer(info)
            
        holder = (leng_mask == 0).sum(-1)
        
        # output_name_nnlvl is not necessarily to be stored in the 
        return self.output_name_nnlvl, {'holder': holder, 'info': info}
    

## Config

In [23]:
def get_reducer_para(nn_name, nn_para, input_size, output_size, postprocess):
    reducer_layer_para = {}
    reducer_layer_para['nn_type'] = 'reducer'
    reducer_layer_para['nn_name'] = nn_name
    reducer_layer_para['nn_para'] = nn_para
    reducer_layer_para['input_size'] = input_size
    reducer_layer_para['output_size'] = output_size
    reducer_layer_para['postprocess'] = postprocess
    return reducer_layer_para

# Merger: BasicNN Name

## Merger-Merge

In [19]:
import torch

class MergeLayer(torch.nn.Module):
    def __init__(self):
        super(MergeLayer, self).__init__()
    
    def forward(self, tensor_list, order = -2):
        info = torch.cat([i.unsqueeze(order) for i in tensor_list], order)
        return info   

# Merger: BasicNN Type

## Module

In [20]:
import torch
# from .merge import MergeLayer

class Merger_Layer(torch.nn.Module):
    def __init__(self, input_names_nnlvl, output_name_nnlvl, merger_layer_para):
        super(Merger_Layer, self).__init__()
        
        # the input_names_nnlvl
        self.input_names_nnlvl = input_names_nnlvl
        # output_name should be generated from the input_names
        self.output_name_nnlvl = output_name_nnlvl
        
        self.input_size = merger_layer_para['input_size']
        self.output_size = merger_layer_para['output_size']
        
        # Part 1: NN
        nn_name = merger_layer_para['nn_name']
        nn_para = merger_layer_para['nn_para']
        
        if nn_name.lower() == 'merger':
            self.merger = MergeLayer()
        else:
            raise ValueError(f'The NN "{nn_name}" is not available')
        
        # Part 2: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in merger_layer_para['postprocess'].items():
            if method == 'activator':
                activator = config
                if activator.lower() == 'relu': 
                    self.postprocess[method] = torch.nn.ReLU()
                elif activator.lower() == 'tanh': 
                    self.postprocess[method] = torch.nn.Tanh()
                elif activator.lower() == 'gelu':
                    self.postprocess[method] = torch.nn.GELU()
            elif method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.output_size, **config)
                
    def forward(self, input_names_nnlvl, INPUTS_TO_INFODICT):
        
        INPUTS = {k:v for k, v in INPUTS_TO_INFODICT.items() if k in input_names_nnlvl}

        # (1) holder
        holder = self.merger([data['holder'] for fld, data in INPUTS.items()], -1)
        
        # (2) merge data
        info = self.merger([data['info'] for fld, data in INPUTS.items()], -2)
        
        # (3) post-process
        for name, layer in self.postprocess.items():
            info = layer(info)

        return self.output_name_nnlvl, {'holder': holder, 'info': info}

## Config

In [21]:
def get_merger_para(nn_name, nn_para, input_size, output_size, postprocess):
    para = {}
    para['nn_type'] = 'merger'
    para['nn_name'] = nn_name
    para['nn_para'] = nn_para
    para['input_size'] = input_size
    para['output_size'] = output_size
    para['postprocess'] = postprocess
    return para